<a href="https://colab.research.google.com/github/alurabhinava/IITM_BS_in_DataScience/blob/MachineLearningPractice/MLP_Week3_GA_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
df = pd.read_csv('/content/drive/MyDrive/MLP/DataPreprocessingGraded_dataset.csv')

In [5]:
df

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,?,?,4000.0,35.0,NEGATIVE,YES
3,?,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO
...,...,...,...,...,...,...
743,23.0,2.0,500.0,38.0,NEGATIVE,NO
744,21.0,2.0,500.0,52.0,NEGATIVE,NO
745,23.0,3.0,750.0,62.0,NEGATIVE,NO
746,39.0,1.0,250.0,39.0,NEGATIVE,NO


In [30]:
df.replace('?', np.nan, inplace=True)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [7]:
estimators = [('imputer', SimpleImputer(strategy='mean'),[0,1]), ('scaler', StandardScaler())]

In [9]:
df[:2]

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES


In [12]:
estimator = [('imputer', SimpleImputer(strategy='mean'), [0,1])]

In [14]:
pipe = Pipeline(steps=estimator)

In [15]:
pipe.fit(df)

ValueError: too many values to unpack (expected 2)

In [31]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion

In [38]:
num_pipeline = Pipeline([('selector', ColumnTransformer([('select_first_2','passthrough', slice(0,4))])),
                        ('Imputer[0,1]',ColumnTransformer([('imputer_0', SimpleImputer(strategy='mean'),[0]),
                                           ('imputer_1', SimpleImputer(strategy='mean'),[1]),
                                            ('select_last_2','passthrough', slice(2,4))])),
                        ('scaler', StandardScaler())
                        ])
cat_pipeline = ColumnTransformer([('Label_OrdinalEncoder', OrdinalEncoder(), [4])])
full_pipeline = FeatureUnion(transformer_list=[('num_pipeline', num_pipeline),
                                               ('cat_pipeline', cat_pipeline)])

In [39]:
from sklearn import set_config
set_config(display='diagram')
full_pipeline

FeatureUnion(transformer_list=[('num_pipeline',
                                Pipeline(steps=[('selector',
                                                 ColumnTransformer(transformers=[('select_first_2',
                                                                                  'passthrough',
                                                                                  slice(0, 4, None))])),
                                                ('Imputer[0,1]',
                                                 ColumnTransformer(transformers=[('imputer_0',
                                                                                  SimpleImputer(),
                                                                                  [0]),
                                                                                 ('imputer_1',
                                                                                  SimpleImputer(),
                                                                                  [1]),
                                                                                 ('select_last_2',
                                                                                  'passthrough',
                                                                                  slice(2, 4, None))])),
                                                ('scaler', StandardScaler())])),
                               ('cat_pipeline',
                                ColumnTransformer(transformers=[('Label_OrdinalEncoder',
                                                                 OrdinalEncoder(),
                                                                 [4])]))])

In [29]:
df.V5.unique()

array(['NEGATIVE'], dtype=object)

In [40]:
df_transform = full_pipeline.fit_transform(df)

In [41]:
df_transform[:5]

array([[-9.38169390e-01,  7.70986653e+00,  7.62334626e+00,
         2.61563344e+00,  0.00000000e+00],
       [-1.18627754e+00,  1.30454949e+00,  1.28273826e+00,
        -2.57880900e-01,  0.00000000e+00],
       [ 0.00000000e+00, -1.53758496e-16,  1.79684161e+00,
         2.94705348e-02,  0.00000000e+00],
       [ 0.00000000e+00,  2.51636623e+00,  2.48231275e+00,
         4.39972584e-01,  0.00000000e+00],
       [-1.06222347e+00,  3.20883294e+00,  3.16778388e+00,
         1.75357914e+00,  0.00000000e+00]])

In [43]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=0.1)
data_new = vt.fit_transform(df_transform)
data_new.shape

(748, 4)

In [66]:
oe = OrdinalEncoder()
Target_oe = oe.fit_transform(df[['Target']])

In [50]:
Target_oe.shape

(748, 1)

In [51]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=2)
selector.fit(df_transform, Target_oe)
print(selector.support_)
print(selector.ranking_)

[ True False  True False False]
[1 3 1 2 4]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [56]:
df_transform.shape

(748, 5)

In [57]:
Target_oe.shape

(748, 1)

In [59]:
Target_oe = np.array(Target_oe)

In [64]:
Target_oe = Target_oe.ravel().shape

In [70]:
from sklearn.feature_selection import SequentialFeatureSelector
estimator = LogisticRegression()
sfs = SequentialFeatureSelector(estimator, n_features_to_select=2)
sfs.fit_transform(df_transform, Target_oe.ravel())
print(sfs.get_support())

[False  True False  True False]


In [10]:
si = SimpleImputer(strategy='mean')
si.fit(df)

SyntaxError: positional argument follows keyword argument (<ipython-input-10-f013ff13fc64>, line 1)